George Mason University, Fairfax, VA 22030

Department of Data Analytics Engineering

Course : DAEN 690

Team name: Team Raiders

Project: Transport Decision Making

Instructor : Dr Issac K Gang

Team Members:

Rakesh Subramani Kaleeshwaran
Bhavana Emmadi
Siri Chandana Keerthipati
Aishwarya Sharma

Importing the necessary python libraries.

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import bs4 as bs
from datetime import timedelta
import urllib.request

Reading the given CSV files to the variables.

Storing the data as pandas dataframe as shown below.

In [2]:
data_1 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/Incidents.csv")

In [3]:
data_2 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/Units.csv")

In [4]:
data_3 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/Patients.csv")


In [5]:
data_4 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/Procedures.csv")


In [6]:
data_5 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/Medications.csv")


In [7]:
data_6 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/first_due.csv")


In [8]:
data_7 = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/Facilities_latest.csv")

So, from the given relationships between the tables, performing the joins and preparing the final data which 
highlights only the records which are important for the analysis of the transport decision making.

The data were made to bring to the level of per patient basis since, the transport decision making is involved with
making decisions for each patient transported.

Dropping the unnecessary columns from the data tables which doesn't give any use (might be error)

In units table:

In [10]:
indx = data_2[data_2.PrimaryKey.isnull()].index
print(indx)
data_2 = data_2.drop(indx)
data_2 = data_2.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'], axis = 1)

Int64Index([280156, 280157, 280158, 280159, 280160, 280161, 280162, 280163,
            280164, 280165,
            ...
            284744, 284745, 284746, 284747, 284748, 284749, 284750, 284751,
            284752, 284753],
           dtype='int64', length=4598)


In Patients table dropping the records which were represents those records where pateints were not transported to
hospital by the provider.

In [11]:
indx2 = data_3[data_3.TransportHospitalName.isnull()].index
print(indx2)
data_3 = data_3.drop(indx2)

Int64Index([    86,    264,    313,    691,    890,   1308,   1340,   1341,
              1719,   2018,
            ...
            121572, 122210, 122265, 122424, 123302, 123560, 124631, 124638,
            125032, 125108],
           dtype='int64', length=481)


Starting the merge and preparing the data from the bottom-up tables which are medications given to the patients

so, before merging preparing the data such a way that there can be multiple medications to a patient can be preserved as the same and divided with '|' in between.

In [12]:
data_5.MedicationGiven[data_5.MedicationGiven.isnull()] = "None given"
data_5.isnull().sum()
data_5.MedicationGiven = data_5.MedicationGiven.astype('object')
data_5.MedicationGiven.dtype
data5_sub1 = data_5.groupby('PatientID')['MedicationGiven'].apply('|'.join).reset_index()

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_2301/1988471468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_5.MedicationGiven[data_5.MedicationGiven.isnull()] = "None given"


Preparing another sub dataframe which takes in the unique number of medications given and the unique number of the isals calls for the medications given per patient.

In [13]:
data5_sub2 = pd.DataFrame(data_5.groupby('PatientID').agg({"MedicationPrimaryKey": "nunique","isALS_LOC": "nunique"})).reset_index()

In [14]:
data5_sub2
data5_sub2.isALS_LOC[data5_sub2.isALS_LOC > 1] = "1"
data5_sub2.isALS_LOC[data5_sub2.isALS_LOC == 1] = "0"

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_2301/3189233107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5_sub2.isALS_LOC[data5_sub2.isALS_LOC == 1] = "0"


In [36]:
data5_sub2[data5_sub2.isALS_LOC == '1'][10:20]

,PatientID,MedicationPrimaryKey,isALS_LOC
100,643400,7,1
103,643471,2,1
108,643592,5,1
115,643766,4,1
120,643829,4,1
122,643903,3,1
131,644030,3,1
136,644128,4,1
140,644194,3,1
143,644224,2,1


In [37]:
data_5[100:110]

,PatientID,MedicationPrimaryKey,MedicationGiven,isALS_LOC
100,1001723,409982,Ondansetron (Zofran),1
101,1001750,409983,Diphenhydramine (Benadryl),1
102,1001750,409984,Methylprednisolone (Solu-Medrol),1
103,1001759,409985,Oxygen,0
104,1001759,409986,"Acetylsalicylic Acid (Aspirin, ASA)",1
105,1001798,409989,Normal saline,1
106,1001802,409990,Dextrose 10 % (D10),1
107,1001850,409993,Naloxone (Narcan),0
108,1001850,409994,Midazolam (Versed),1
109,1001879,409995,Midazolam (Versed),1


In [ ]:
merging the dataframes that were created above as a replacement for the medication data given and renaming the columns.

In [15]:
ll_data1 = pd.merge(data5_sub1, data5_sub2, on ='PatientID') 
ll_data1.shape
ll_data1.rename(columns = {'MedicationPrimaryKey':'Total_Number_of_medications_given', 'isALS_LOC' : 'Medication_given_ALS?'}, inplace = True)

In [24]:
ll_data1[60:80]

,PatientID,MedicationGiven,Total_Number_of_medications_given,Medication_given_ALS?
60,642258,Normal saline|Oxygen|Ipratropium Bromide(Atrov...,4,1
61,642269,Normal saline|Normal saline,2,0
62,642273,Fentanyl Citrate (Sublimaze) |Fentanyl Citrate...,2,0
63,642276,Normal saline|Ondansetron (Zofran),2,0
64,642283,Normal saline,1,0
65,642291,Oxygen|Ipratropium Bromide(Atrovent)|Albuterol...,3,1
66,642391,"Acetylsalicylic Acid (Aspirin, ASA) |Nitroglyc...",2,0
67,642400,Normal saline,1,0
68,642422,Ipratropium Bromide(Atrovent)|Albuterol (Vento...,2,0
69,642475,Ondansetron (Zofran),1,0


Taking the procedures performed on the patients, and bringing it to per patient level as we did for medications.

For bringing the records to the per patient level.

In [25]:
data4_sub1 = data_4.groupby('PatientID')['ProcedurePerformed'].apply('|'.join).reset_index()

Preparing another sub dataframe which takes in the unique number of procedures performed 
and the unique number of the isals calls for the procedures given per patient.

In [26]:
data4_sub2 = pd.DataFrame(data_4.groupby('PatientID').agg({"ProcedurePrimaryKey": "nunique", "isALS_LOC": "nunique"})).reset_index()
data4_sub2.isALS_LOC[data4_sub2.isALS_LOC > 1] = "1"
data4_sub2.isALS_LOC[data4_sub2.isALS_LOC == 1] = "0"

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_2301/753221165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4_sub2.isALS_LOC[data4_sub2.isALS_LOC == 1] = "0"


merging the dataframes that were created above as a replacement for 
the procedure data given and renaming the columns.

In [17]:
ll_data2 = pd.merge(data4_sub1, data4_sub2, on ='PatientID')
ll_data2.rename(columns = {'ProcedurePrimaryKey':'Total_Number_of_procedures_performed', 'isALS_LOC' : 'Procedure_given_ALS?'}, inplace = True)

Merging the patients data with that of procedures performed data that was prepared as given above.

where the left table is the patients data and the right data is the prepared procedures data.

In [18]:
mm = pd.merge(data_3, ll_data2, on='PatientID', how = 'left')

In [19]:
# Run the code below to check the result of the joining the data that was performed above.
# mm.to_csv("../mm.csv")

Merging the patients data with that of medications given data that was prepared as given above.

where the left table is the patients data and the right data is the prepared medications data.

In [20]:
mmm = pd.merge(data_3, ll_data1, on='PatientID', how = 'left')

Dropping the columns that are common to both the tables.

In [21]:
mmm = mmm.drop(['UnitTransportedDT','UnitTransportedArrivalDT','PrimaryKey','TransportUnitID','TransportLOC','TransportHospitalCode','TransportHospitalName','ReasonForChoosingHospital','PatientAge','PrimaryImpression','SecondaryImpression','PatientDisposition'], axis =1)

In [22]:
# Run the below code to get the resultant dataframe into a csv 
# mmm.to_csv("../mmm.csv")

Merging both the resulting dataframes from the joins that were performed above with respect to the patients data.
and performing a outer join. 

In [23]:
mmmm = pd.merge(mm, mmm, on='PatientID', how ='outer')

Merging the data prepared in the above with the units data in given.

Here the left table is the units and right is the resultant data from the above.
We are joining the data with a left outer join.

In [24]:
mak3 = pd.merge(data_2, mmmm, on='PrimaryKey', how ='left')

In [31]:
mak3.TransportHospitalName.isna().sum()
data_3.TransportHospitalName.isna().sum()

data_2.columns

Index(['PrimaryKey', 'ResponseUnitID', 'UnitPrimaryActionTaken',
       'UnitOtherActionsTaken', 'UnitStation', 'UnitType', 'UnitArrivalOrder'],
      dtype='object')

In [25]:
# Run the below code to get the resultant dataframe into a csv 
# mak3.to_csv("../mak3.csv")

Joining the Incidents data with the other resultant data with a inner join.
Where the left table is the incident data and the right table is the resultant data.

In [26]:
final_md2 = pd.merge(data_1, mak3, on='PrimaryKey', how ='inner')
final_md2

,PrimaryKey,CallConfirmedDT,ShiftDay,TourOfShift,IncidentFirstDue,InitialIncidentType,DispatchedIncidentType,ArrivedIncidentType,FinalIncidentType,FinalIncidentTypeDescription,...,PatientAge,PrimaryImpression,SecondaryImpression,PatientDisposition,ProcedurePerformed,Total_Number_of_procedures_performed,Procedure_given_ALS?,MedicationGiven,Total_Number_of_medications_given,Medication_given_ALS?
0,9804095125,07/01/18 00:05:40,B,1,409.0,SHOTF,SHOTF,SHOTF,SHOTF,ASSAULT W/WEAPON - SHOOTING,...,10 - 19,"Injury - Wrist, Hand, or Fingers (S69.9)","""Not Applicable""",Treated & Transported,NaN,NaN,NaN,NaN,NaN,NaN
1,9804095125,07/01/18 00:05:40,B,1,409.0,SHOTF,SHOTF,SHOTF,SHOTF,ASSAULT W/WEAPON - SHOOTING,...,10 - 19,GI/GU - Pelvic or Perineal Pain (R10.2),"""Pain - Not Elsewhere Mentioned Sudden Onset ...",Treated & Transported,IV Start - Extremity Vein (arm or leg)|IV Star...,2.0,0,NaN,NaN,NaN
2,9804095125,07/01/18 00:05:40,B,1,409.0,SHOTF,SHOTF,SHOTF,SHOTF,ASSAULT W/WEAPON - SHOOTING,...,10 - 19,"Injury - Wrist, Hand, or Fingers (S69.9)","""Not Applicable""",Treated & Transported,NaN,NaN,NaN,NaN,NaN,NaN
3,9804095125,07/01/18 00:05:40,B,1,409.0,SHOTF,SHOTF,SHOTF,SHOTF,ASSAULT W/WEAPON - SHOOTING,...,10 - 19,GI/GU - Pelvic or Perineal Pain (R10.2),"""Pain - Not Elsewhere Mentioned Sudden Onset ...",Treated & Transported,IV Start - Extremity Vein (arm or leg)|IV Star...,2.0,0,NaN,NaN,NaN
4,9804095125,07/01/18 00:05:40,B,1,409.0,SHOTF,SHOTF,SHOTF,SHOTF,ASSAULT W/WEAPON - SHOOTING,...,10 - 19,"Injury - Wrist, Hand, or Fingers (S69.9)","""Not Applicable""",Treated & Transported,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293494,12354449085,12/31/20 23:45:44,A,1,444.0,MEDICAL,MEDICAL,MEDICAL,MEDICAL,MEDICAL EMERGENCY,...,10 - 19,Neuro - Altered Mental Status / Level of Consc...,Abuse of Alcohol - Intoxication (F10.92),Treated & Transported,NaN,NaN,NaN,NaN,NaN,NaN
293495,12354449085,12/31/20 23:45:44,A,1,444.0,MEDICAL,MEDICAL,MEDICAL,MEDICAL,MEDICAL EMERGENCY,...,10 - 19,Neuro - Altered Mental Status / Level of Consc...,Abuse of Alcohol - Intoxication (F10.92),Treated & Transported,NaN,NaN,NaN,NaN,NaN,NaN
293496,12354449085,12/31/20 23:45:44,A,1,444.0,MEDICAL,MEDICAL,MEDICAL,MEDICAL,MEDICAL EMERGENCY,...,10 - 19,Neuro - Altered Mental Status / Level of Consc...,Abuse of Alcohol - Intoxication (F10.92),Treated & Transported,NaN,NaN,NaN,NaN,NaN,NaN
293497,12354229090,12/31/20 23:47:48,A,1,422.0,ALS,ALS,ALS,ALS,ALS EMERGENCY,...,70 - 79,Pain - Chest (presumed non-cardiac) (R07.89),Not Applicable,Treated & Transported,CV - ECG - 12 Lead Obtained,1.0,0,NaN,NaN,NaN


In [27]:
final_md2.to_csv("/Volumes/Personal/DAEN 690 - Project/Dataset/untitled folder/final_merged.csv")